In [23]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [20]:
path = "/home/kodolamacz/PycharmProjects/Python_w4/Pandas - bootcamp/nycflights13_airlines.csv.gz"
path1 = "/home/kodolamacz/PycharmProjects/Python_w4/Pandas - bootcamp/nycflights13_airports.csv.gz"
path2 = "/home/kodolamacz/PycharmProjects/Python_w4/Pandas - bootcamp/nycflights13_flights.csv.gz"
path3 = "/home/kodolamacz/PycharmProjects/Python_w4/Pandas - bootcamp/nycflights13_planes.csv.gz"
path4 = "/home/kodolamacz/PycharmProjects/Python_w4/Pandas - bootcamp/nycflights13_weather.csv.gz"

In [36]:
planes = pd.read_csv(path3,comment='#')
airplanes = pd.read_csv(path,comment='#')
flights = pd.read_csv(path2,comment='#')
airports = pd.read_csv(path1,comment='#')
weather = pd.read_csv(path4,comment='#')

#### Creating db

In [37]:
db_name = 'nycflights13.db'
con = sqlite3.connect(os.path.join(os.getcwd(),db_name)) #

planes.to_sql('planes',con)
planes.to_sql('airplanes',con)
planes.to_sql('flights',con)
planes.to_sql('airports',con)
planes.to_sql('weather',con)

### Doing same ops in pandas as in SQL


In [39]:
sql_result = pd.read_sql_query("SELECT DISTINCT engine FROM planes",con)
sql_result

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


In [44]:
pd.DataFrame(planes['engine'].unique(),columns=['engine'])

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


In [50]:
sql_result2 = pd.read_sql_query('''SELECT DISTINCT type, manufacturer, model, seats, 
                                engine FROM planes''',con)
sql_result2

In [57]:
planes[['type', 'manufacturer', 'model', 'seats', 'engine']].drop_duplicates().reset_index(drop=True).head()

,type,manufacturer,model,seats,engine
0,Fixed wing multi engine,EMBRAER,EMB-145XR,55,Turbo-fan
1,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,182,Turbo-fan
2,Fixed wing multi engine,EMBRAER,EMB-145LR,55,Turbo-fan
3,Fixed wing multi engine,BOEING,737-824,149,Turbo-fan
4,Fixed wing multi engine,BOEING,767-332,330,Turbo-fan


In [83]:
sql_result3 = pd.read_sql_query('''SELECT COUNT(*), engine FROM planes GROUP BY engine''',con)
sql_result3

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


In [ ]:
##### 1st approach
df0 = planes['engine'].value_counts()
##### 2nd approach
df1 = planes.groupby('engine').count()['manufacturer'].reset_index()[['manufacturer','engine']]
df1.columns=(['count','engine'])
df1

In [85]:
df0

Turbo-fan        2750
Turbo-jet         535
Reciprocating      28
Turbo-shaft         5
4 Cycle             2
Turbo-prop          2
Name: engine, dtype: int64

In [90]:
sql_result4 = pd.read_sql('''SELECT COUNT(*),engine,type FROM planes GROUP BY engine, type''',con)
sql_result4

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


In [88]:
df2 = pd.DataFrame(planes.groupby(['engine','type']).agg('count')['engines'])
df2.columns  = ['Count']
df2

Count
engine        type                           
4 Cycle       Fixed wing single engine      2
Reciprocating Fixed wing multi engine       5
              Fixed wing single engine     23
Turbo-fan     Fixed wing multi engine    2750
Turbo-jet     Fixed wing multi engine     535
Turbo-prop    Fixed wing multi engine       2
Turbo-shaft   Rotorcraft                    5

In [93]:
sql_result5 = pd.read_sql('''SELECT MIN(year),AVG(year), MAX(year), engine, manufacturer 
                            FROM planes GROUP BY engine, manufacturer''',con)
sql_result5.head()

,MIN(year),AVG(year),MAX(year),engine,manufacturer
0,1975.0,1975.0,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.0,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L


In [98]:
df5 = planes[['year','engine','manufacturer']].groupby(['engine','manufacturer']).agg(['min','max','mean'])
df5.columns = ['min(year)','max(year)','mean(year)']
df5.reset_index()

,engine,manufacturer,min(year),max(year),mean(year)
0,4 Cycle,CESSNA,1975.0,1975.0,1975.000000
1,4 Cycle,JOHN G HESS,NaN,NaN,NaN
2,Reciprocating,AMERICAN AIRCRAFT INC,NaN,NaN,NaN
3,Reciprocating,AVIAT AIRCRAFT INC,2007.0,2007.0,2007.000000
4,Reciprocating,BARKER JACK L,NaN,NaN,NaN
5,Reciprocating,CESSNA,1959.0,1983.0,1971.142857
6,Reciprocating,CIRRUS DESIGN CORP,2007.0,2007.0,2007.000000
7,Reciprocating,DEHAVILLAND,1959.0,1959.0,1959.000000
8,Reciprocating,DOUGLAS,1956.0,1956.0,1956.000000
9,Reciprocating,FRIEDEMANN JON,2007.0,2007.0,2007.000000


In [99]:
sql_result6 = pd.read_sql('''SELECT * FROM planes WHERE speed IS NOT NULL''',con)
sql_result6

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


In [148]:
planes[planes['speed'].notnull()].reset_index(drop=True)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


In [127]:
sql_result7 = pd.read_sql('''SELECT tailnum FROM planes WHERE year >=2010''',con)
sql_result7.head()

,tailnum
0,N127UW
1,N128UW
2,N150UW
3,N151UW
4,N152UW


In [150]:
pd.DataFrame(planes[planes['year']>=2010]['tailnum']).reset_index(drop=True).head()

,tailnum
0,N127UW
1,N128UW
2,N150UW
3,N151UW
4,N152UW


In [152]:
sql_result7 = pd.read_sql('''SELECT tailnum FROM planes WHERE seats BETWEEN 100 and 200 LIMIT 20 ''',con)
sql_result7

,tailnum
0,N102UW
1,N103US
2,N104UW
3,N105UW
4,N107US
5,N108UW
6,N109UW
7,N110UW
8,N111US
9,N11206


In [159]:
pd.DataFrame(planes[(planes['seats'] >=100)&(planes['seats'] <=200)]['tailnum'].reset_index(drop = True).head(20))

,tailnum
0,N102UW
1,N103US
2,N104UW
3,N105UW
4,N107US
5,N108UW
6,N109UW
7,N110UW
8,N111US
9,N11206


In [180]:
sql_result8 = pd.read_sql('''SELECT * FROM planes WHERE manufacturer IN ("BOEING","AIRBUS", "EMBRAER")''',con).reset_index(drop=True)
sql_result8.head()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
1,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,None,Turbo-fan
2,10,N11106,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
3,11,N11107,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
4,12,N11109,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan


In [168]:
planes[planes['manufacturer'].isin(["BOEING","AIRBUS", "EMBRAER"])].head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan
10,N11106,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
11,N11107,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
12,N11109,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan


In [171]:
sql_result9 = pd.read_sql('''SELECT * FROM planes WHERE manufacturer IN ("BOEING",
                            "AIRBUS", "EMBRAER") AND seats>300''',con)
sql_result9.head()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan


In [175]:
manufat = planes['manufacturer'].isin(["BOEING","AIRBUS", "EMBRAER"])
seat_300 = planes['seats'] > 300

planes[manufat&seat_300].reset_index(drop=True).head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
1,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
2,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
3,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
4,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan


In [178]:
sql_result10 = pd.read_sql('''SELECT manufacturer, COUNT(*) FROM planes WHERE seats >
200 GROUP BY manufacturer''',con)
sql_result10.head()

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [183]:

pd.DataFrame(planes[planes['seats'] > 200].groupby('manufacturer').agg('count')['seats'])

,seats
manufacturer,
AIRBUS,66
AIRBUS INDUSTRIE,4
BOEING,225


In [184]:
sql_result11 = pd.read_sql('''SELECT manufacturer, COUNT(*) FROM planes GROUP BY
                                manufacturer HAVING COUNT(*) > 10''',con)
sql_result11

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14


In [201]:
planes_manuf =  planes.groupby('manufacturer').size() #size is the method that shpuld be used 
pd.DataFrame(planes_manuf[planes_manuf > 10])

,0
manufacturer,
AIRBUS,336
AIRBUS INDUSTRIE,400
BOEING,1630
BOMBARDIER INC,368
EMBRAER,299
MCDONNELL DOUGLAS,120
MCDONNELL DOUGLAS AIRCRAFT CO,103
MCDONNELL DOUGLAS CORPORATION,14


In [203]:
sql_result12 = pd.read_sql('''SELECT manufacturer, COUNT(*) FROM planes WHERE seats >
                            200 GROUP BY manufacturer HAVING COUNT(*) > 10''',con)

sql_result12

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [207]:
planes_manuf = planes[planes['seats']>200].groupby('manufacturer').size()
pd.DataFrame(planes_manuf[planes_manuf>10])

,0
manufacturer,
AIRBUS,66
BOEING,225


In [211]:
sql_result13 = pd.read_sql('''SELECT manufacturer, COUNT(*) AS howmany FROM planes
                            GROUP BY manufacturer ORDER BY howmany''',con)
sql_result13.head()

,manufacturer,howmany
0,AGUSTA SPA,1
1,AVIAT AIRCRAFT INC,1
2,AVIONS MARCEL DASSAULT,1
3,BARKER JACK L,1
4,CANADAIR LTD,1


In [218]:
df = pd.DataFrame(planes.groupby('manufacturer').size().sort_values()).head()
df.columns = ['howmany']
df

,howmany
manufacturer,
AGUSTA SPA,1
DOUGLAS,1
DEHAVILLAND,1
CIRRUS DESIGN CORP,1
HURLEY JAMES LARRY,1


In [220]:
sql_result14 = pd.read_sql('''SELECT manufacturer, COUNT(*) AS howmany FROM planes
                            GROUP BY manufacturer ORDER BY howmany DESC LIMIT 10''',con)
sql_result14

,manufacturer,howmany
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14
8,CANADAIR,9
9,CESSNA,9


In [221]:
df = pd.DataFrame(planes.groupby('manufacturer').size().sort_values(ascending = False)).head()
df.columns = ['howmany']
df

,howmany
manufacturer,
BOEING,1630
AIRBUS INDUSTRIE,400
BOMBARDIER INC,368
AIRBUS,336
EMBRAER,299


In [314]:
## 1st approach 
a = np.array(np.repeat((True,False),50))
np.random.shuffle(a)

## 2nd approach
airports.sample(frac = 0.05).head()

,faa,name,lat,lon,alt,tz,dst,tzone
973,NOW,Port Angeles Cgas,48.141481,-123.414075,13,-8,A,America/Los_Angeles
469,FBG,Fredericksburg Amtrak Station,38.298416,-77.456875,130,-5,A,America/New_York
776,LCH,Lake Charles Rgnl,30.126112,-93.223335,15,-6,A,America/Chicago
590,HIF,Hill Afb,41.123939,-111.973039,4789,-7,A,America/Denver
1390,WAL,Wallops Flight Facility,37.940194,-75.466389,40,-5,A,America/New_York


In [231]:
A = airports.iloc[:10,:]
B = airports.iloc[6:15,:]

,faa,name,lat,lon,alt,tz,dst,tzone
6,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
7,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
8,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
9,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles
10,0W3,Harford County Airport,39.566838,-76.202403,409,-5,A,America/New_York
11,10C,Galt Field Airport,42.402889,-88.375111,875,-6,U,America/Chicago
12,17G,Port Bucyrus-Crawford County Airport,40.781556,-82.974806,1003,-5,A,America/New_York
13,19A,Jackson County Airport,34.175864,-83.561597,951,-5,U,America/New_York
14,1A3,Martin Campbell Field Airport,35.015806,-84.346833,1789,-5,A,America/New_York


In [245]:
#union all
pd.concat([A,B]) 

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593,-5,A,America/New_York
6,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
7,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
8,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
9,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [246]:
### union
pd.concat([A,B]).drop_duplicates()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593,-5,A,America/New_York
6,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
7,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
8,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
9,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [342]:
## intersect
A.iloc[A.index[A.index.intersection(B.index)],:]
### with merge
A.merge(B,how='inner')


,faa,name,lat,lon,alt,tz,dst,tzone
0,0G6,Williams County Airport,41.467306,-84.506778,730,-5,A,America/New_York
1,0G7,Finger Lakes Regional Airport,42.883565,-76.781232,492,-5,A,America/New_York
2,0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5,U,America/New_York
3,0S9,Jefferson County Intl,48.053809,-122.810644,108,-8,A,America/Los_Angeles


In [347]:
## except
exc = A.isin(B)
A_exc = A[~exc]
A[~A_exc['faa'].isna()]

## 2 nd approach
A.drop(B.index,errors='ignore')

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
5,0A9,Elizabethton Municipal Airport,36.371222,-82.173417,1593,-5,A,America/New_York


In [326]:
exc = B.isin(A)
B_exc = B[~exc]
B[~B_exc['faa'].isna()]

,faa,name,lat,lon,alt,tz,dst,tzone
10,0W3,Harford County Airport,39.566838,-76.202403,409,-5,A,America/New_York
11,10C,Galt Field Airport,42.402889,-88.375111,875,-6,U,America/Chicago
12,17G,Port Bucyrus-Crawford County Airport,40.781556,-82.974806,1003,-5,A,America/New_York
13,19A,Jackson County Airport,34.175864,-83.561597,951,-5,U,America/New_York
14,1A3,Martin Campbell Field Airport,35.015806,-84.346833,1789,-5,A,America/New_York


In [334]:
flght_plns = flights.merge(planes,on='tailnum',how='left')

In [332]:
flght_arprt = flights.merge(airports,on='tailnum',how='left')

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan


In [338]:
airports.head(5)

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York


In [337]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


In [339]:
planes.head(1)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
